# VIVA: Visualization of Variants

## VIVA is a tool for visualizing variant genotype and read depth (coverage) data from variant call format (VCF) files. 

### Getting Started

Enter your VCF filename below and choose your plotting options.

Then run the entire notebook. Plots will be saved in the directory you keep your VIVA.ipynb in, or in the directory you specify in the plotting options.


### Enter You VCF filename

Enter filename or file path of a VCF file to visualize. 

In [1]:
vcf_filename = "/Users/George/.julia/dev/VariantVisualization/test/test_files/test_4X_191.vcf" ;
println("loaded filename")

loaded filename


### Variant Selection

Select variant record rows of your VCF file to visualize.

*Important Note* Variant selection is recommending when visualizing large VCF files as plotting capability is limited by user's computing power.

In [2]:
genomic_range = "chr4:0-500000000" #specify genomic range of variants to select in the format chr4:0-560000000
pass_filter = "true" #if "true", selects variants with Filter Status = PASS
positions_list = "test/test_files/positions_list_no_strings.csv" #include path and filename to list of variant positions to selects
; 
println("loaded options")

loaded options


### Sample Grouping and Selection

Add sample metadata to heatmap plots and group sample columns by metadata traits. This feature is useful for identifying batch effects or differential variation between samples. 

Additionally, you may select unique samples to include in heatmap and average read depth scatter plots. 

*Note* If you would like to group samples using a metadata table in combination with the sample selection feature, the metadata table must only include metadata for the samples to be selected.

In [3]:
group_samples = "/Users/George/.julia/dev/VariantVisualization/test/test_files/sample_metadata_matrix.csv","case,control" #include path or filename to matrix to group by, as well as trait to group by according to the manual
select_samples = "" #include path and filename to list of sample names
;
println("loaded options")

loaded options


### Plotting Options

In [4]:
heatmaps = "genotype,read_depth" #make default both - genotype, read_depth, or genotype,read_depth
read_depth_scatter_plots = "" 
y_axis_label_option = "chromosomes"#defaults to "chromosome labels". If "positions", each variant position is labeled on y axis. If "none", no y-axis ticklabels are plotted.
x_axis_label_option = "" #if true, sample ids are labelled on x axis labels
;
println("loaded options")

loaded options


### Saving Options

In [5]:
heatmap_title = ""
save_format = "html" #Options include html, pdf, svg, eps, png
output_directory = ""
num_array = "" #if "true", saves labeled number array of filtered heatmap input
save_remotely = "true" #if true, html outputs are shareable but viewer must have internet connection to open 
;
println("loaded options")

loaded options


### Run the Analysis
##### Now that you've entered your settings, click 'Run All' in the Cell dropdown in the notebook menubar to run the whole notebook and run the VIVA options.
Do not alter the code in the cells below. Your files will save to the output directory in which the notebook file (.ipynb) is saved, unless you chose a non-default output directory in the settings.

In [6]:
### DO NOT ALTER THE CODE IN THIS CELL ###
### Navigate to `Cell` tab in the menubar and click `Run All` to generate plots ###

#Passing settings to VIVA. Do not adjust.

println("Welcome to VIVA Jupyter Notebook.")
println()
println("Loading packages:")
println()


println("1/5 ArgParse")
using ArgParse

println("2/5 VariantVisualization")
using VariantVisualization

println("3/5 PlotlyJS")
using PlotlyJS

println("4/5 GeneticVariation")
using GeneticVariation

println("5/5 DelimitedFiles")
using DelimitedFiles

variant_filters = genomic_range,pass_filter,positions_list
sample_selection = group_samples,select_samples
plotting_options = heatmaps,read_depth_scatter_plots,y_axis_label_option,x_axis_label_option
saving_options = heatmap_title,save_format,output_directory,num_array,save_remotely

Welcome to VIVA Jupyter Notebook.

Loading packages:

1/5 ArgParse
2/5 VariantVisualization


┌ Info: Recompiling stale cache file /Users/George/.julia/compiled/v1.1/VariantVisualization/v8cR2.ji for VariantVisualization [7f8bf45c-0ad5-53b5-97e6-33c1305e0aa4]
└ @ Base loading.jl:1184
[ Info: Compiling VCF parser...


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/d66afc8b80ce609a2ce630d051df1008d166cdff-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/d66afc8b80ce609a2ce630d051df1008d166cdff-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

3/5 PlotlyJS
4/5 GeneticVariation
5/5 DelimitedFiles


("", "html", "", "", "true")

In [7]:
jupyter_main(vcf_filename,saving_options,variant_filters,sample_selection,plotting_options)
;

Reading /Users/George/.julia/dev/VariantVisualization/test/test_files/test_4X_191.vcf

Selected 5 variants with Filter status: PASS, that match list of chromosome positions of interest, and are within chromosome range: chr4:0-500000000

Finished Filtering.
_______________________________________________

Grouping samples by case,control



data: [
  "heatmap with fields colorbar, colorscale, gridcolor, hoverinfo, showscale, text, transpose, type, x, y, z, zauto, zmax, and zmin",
  "scatter with fields marker, mode, name, text, textfont, textposition, type, x, and y"
]

layout: "layout with fields margin, shapes, title, xaxis, and yaxis"

Saving genotype heatmap

Grouping samples by case,control



data: [
  "heatmap with fields colorbar, colorscale, gridcolor, hoverinfo, showscale, text, transpose, type, x, y, z, zauto, zmax, and zmin",
  "scatter with fields marker, mode, name, text, textfont, textposition, type, x, and y"
]

layout: "layout with fields margin, shapes, title, xaxis, and yaxis"

Saving read depth heatmap
